### 다중언어 개체명인식

- 제로샷 교차 언어 전이 (zero-shot cross lingual switching) 가능
  - 한 언어에서 파인 튜닝된 모델이 훈련없이 다른 모델에 적용 가능하다.
- 코드 스위칭 에 적합
  - 하나의 대화에서 둘이상의 언어나, 사투리등을 바꿈

- XTREME 데이터 셋 이용
    - PAN-X, wikiANN : 교차 언어 데이터 셋
    - LOC, PER, ORG
    - IOB2 포맷
        - i- 동일 개체명의 연속
        - o- 어떤 개체에도 속하지 않음
        - B- 개체명의 시작

In [ ]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
"서브셋 갯수",len(xtreme_subsets)

('서브셋 갯수', 183)

In [85]:
for i,l in enumerate(xtreme_subsets):
    if "PAN" in l:
        print(l.split(".")[-1], end=': ')
        print(l, end=', ')
        if i % 5 == 0:
            print("")

af: PAN-X.af, ar: PAN-X.ar, 
bg: PAN-X.bg, bn: PAN-X.bn, de: PAN-X.de, el: PAN-X.el, en: PAN-X.en, 
es: PAN-X.es, et: PAN-X.et, eu: PAN-X.eu, fa: PAN-X.fa, fi: PAN-X.fi, 
fr: PAN-X.fr, he: PAN-X.he, hi: PAN-X.hi, hu: PAN-X.hu, id: PAN-X.id, 
it: PAN-X.it, ja: PAN-X.ja, jv: PAN-X.jv, ka: PAN-X.ka, kk: PAN-X.kk, 
ko: PAN-X.ko, ml: PAN-X.ml, mr: PAN-X.mr, ms: PAN-X.ms, my: PAN-X.my, 
nl: PAN-X.nl, pt: PAN-X.pt, ru: PAN-X.ru, sw: PAN-X.sw, ta: PAN-X.ta, 
te: PAN-X.te, th: PAN-X.th, tl: PAN-X.tl, tr: PAN-X.tr, ur: PAN-X.ur, 
vi: PAN-X.vi, yo: PAN-X.yo, zh: PAN-X.zh, 

### 언어 선택하기

- 선택 언어
    - PAN-X.en : 영어
    - PAN-X.ko : 한국어
    - PAN-X.ja : 일본어
    - PAN-X.es : 스페인어

In [86]:
from datasets import load_dataset
from collections import defaultdict
from datasets import DatasetDict

#일반적인 불균형 상태 만들기
langs = ["en", "ko", "ja", "es"]
fracs = [0.12, 0.6, 0.8, 0.1]

panx_ch = defaultdict(DatasetDict)

In [90]:
for lang, frac in zip(langs, fracs):
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split]=(
            ds[split]
            .shuffle(seed=42)
            .select(range(int(frac*ds[split].num_rows))))


In [92]:
import pandas as pd

pd.DataFrame({lang : [panx_ch[lang]["train"].num_rows] for lang in langs}, index=["Samples"])

,en,ko,ja,es
Samples,2400,12000,16000,2000


In [93]:
element = panx_ch["ko"]["train"][0]

for k, v in element.items():
    print(k,v)

print("\nfeatures 중 ner-태그 확인")
ner_tags = panx_ch["ko"]["train"].features["ner_tags"].feature
ner_tags

tokens ['《트와일라잇》을', '같이', '찍은', '에디', '가테지', ',', '크리스틴', '스튜어트', ',', '로버트', '패틴슨과는', '매우', '친한사이라고', '한다', '.']
ner_tags [0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 0, 0, 0]
langs ['ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko', 'ko']

features 중 ner-태그 확인


ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [94]:
# NER-태그를 인덱스 에서 태그명 변경 및 추가

panx_ko = panx_ch["ko"].map(lambda x : {"ner_tag_names" :
                                        [ner_tags.int2str(idx)
                                        for idx in x["ner_tags"]]})
panx_ko

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 12000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 6000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tag_names'],
        num_rows: 6000
    })
})

In [95]:
pd.DataFrame([panx_ko["train"][0]["tokens"],
              panx_ko["train"][0]["ner_tag_names"]],
              index=["tokens","ner_tag_name"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,《트와일라잇》을,같이,찍은,에디,가테지,",",크리스틴,스튜어트,",",로버트,패틴슨과는,매우,친한사이라고,한다,.
ner_tag_name,O,O,O,B-PER,I-PER,O,B-PER,I-PER,O,B-PER,I-PER,O,O,O,O


- 결과: 사람 이름에 태그가 붙었다.

### 태그 분포 확인하기

In [96]:
from collections import Counter

split2freqs = defaultdict(Counter)

for split , dataset in panx_ko.items():
    for row in dataset["ner_tag_names"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

split2freqs

defaultdict(collections.Counter,
            {'train': Counter({'LOC': 7097, 'ORG': 5361, 'PER': 4870}),
             'validation': Counter({'LOC': 3579, 'ORG': 2755, 'PER': 2448}),
             'test': Counter({'LOC': 3503, 'ORG': 2584, 'PER': 2557})})

### 다중 언어 트랜스 포머

- 일반 적으로 데이터셋에 여러 언어가 있어도 일반화가 가능하다.
- 다중 언어 트랜스 포머 평가 방법
    1. **en (영어 훈련 후 평가)**: 영어 데이터로 훈련한 후 다른 언어에서 평가.
    2. **each (단일 훈련 및 단일 평가)**: 각 언어별로 별도로 훈련하고 평가.
    3. **all (모든 훈련셋에서 평가)**: 모든 언어 데이터를 사용해 훈련하고 각 언어에서 평가.

### XLM-R(Cross-lingual LM)
- RoBERTa 의 다중 언어 모델 버젼 : XLM-RoBERTa
- 100개 의 언어로 훈련

- **SentencePiece** 토크나이저
    - Unigram(부분 단어 분할 방식) 기반의 인코딩 방식을 이용
    - 특정 언어에 대한 지식없이 텍스트 처리 가능
    - 다국어 말뭉치에 유용하다
    - 다양한 언어 모델과 호환성이 좋다.

In [97]:
from transformers import AutoTokenizer

bertmodel = "bert-base-cased"
xlmrmodel = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bertmodel)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmrmodel) # Sentence Piece 토크나이저

In [98]:
text_test = "Im Your Father"

print(bert_tokenizer.convert_ids_to_tokens(bert_tokenizer(text_test).input_ids))
print(xlmr_tokenizer.convert_ids_to_tokens(xlmr_tokenizer(text_test).input_ids))

['[CLS]', 'I', '##m', 'Your', 'Father', '[SEP]']
['<s>', '▁Im', '▁Your', '▁Father', '</s>']


### 토큰화 파이프라인

1. **정규화 (Normalization)**
   - 텍스트를 일관된 형태로 변환.
   - 예: `'Im Your Father'` → `'im your father'`

2. **사전 토큰화 (Pre-tokenization)**
   - 공백과 구두점을 기준으로 단어로 나눈다..
   - 예: `'im your father'` → `['im', 'your', 'father']`

3. **토크나이저 모델 (Tokenizer Model)**
   - 단어를 고유한 숫자 ID로 바꿈.
   - 예: `['im', 'your', 'father']` → `[125, 52, 482]`

4. **사후처리 (Post-processing)**
   - 시작과 끝을 나타내는 특수 토큰을 추가.
   - 예: `[CLS] im your father [SEP]` → `[0, 125, 52, 482, 1]`

### 트랜스 포머 모델 클래스의 형식
![modelfortask](images/04_01.png)
- `<ModelName>For<Task>` 형식을 띔

### xtreme 데이터 셋 -> 커널 종료 후 아래 코드 부터 다시 실행

In [1]:
from datasets import get_dataset_config_names

# xtreme 데이터셋 이름 확인
subsets = get_dataset_config_names("xtreme")

def find_lan(lan):
    return [j for j in [i for i in subsets if "PAN-X" in i]
                if lan in j][0]

language_names = [find_lan(l) for l in ["ko", "en", "ja", "es"]]
language_names

['PAN-X.ko', 'PAN-X.en', 'PAN-X.ja', 'PAN-X.es']

### 데이터 셋 로드 및 전처리

In [2]:
from collections import defaultdict
from datasets import DatasetDict, load_dataset

panx_ch = defaultdict(DatasetDict)
down_sample_ratios = [0.5, 0.3, 0.2, 0.1] #현실 데이터 와 같이 데이터 불균형 만들기
# 데이터 셋 로드

for l, down_sample_ratio in zip(language_names, down_sample_ratios):
    l_name = (l.split('.')[-1])
    down_sample = load_dataset("xtreme", name=l)

    for split in down_sample:
        # 선택할 데이터 포인트의 수
        num_datas = int(down_sample[split].num_rows * down_sample_ratio)
        panx_ch[l_name][split] = down_sample[split].shuffle(seed=42).select(range(num_datas))

In [3]:
print(f"""각 언어별 데이터 포인트 수 :
{[(j,panx_ch[j].num_rows) for j in [i for i in panx_ch]]}""")

각 언어별 데이터 포인트 수 :
[('ko', {'train': 10000, 'validation': 5000, 'test': 5000}), ('en', {'train': 6000, 'validation': 3000, 'test': 3000}), ('ja', {'train': 4000, 'validation': 2000, 'test': 2000}), ('es', {'train': 2000, 'validation': 1000, 'test': 1000})]


In [4]:
labels = panx_ch["ko"]["train"].features["ner_tags"].feature.names

i2l = {k:v for k,v in enumerate(labels)}
l2i = {v:k for k,v in enumerate(labels)}

print(i2l,"\n",l2i)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'} 
 {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


- 라벨 맵핑하기

In [5]:
ner_tags_str = lambda bs : {"ner_tags_str" :([[i2l[i] for i in b] for b in bs["ner_tags"]])}

for l in ["ko", "en", "ja", "es"]:
    panx_ch[l] = (panx_ch[l]
                    .map(ner_tags_str,
                    batch_size= 100, batched=True))

In [6]:
import pandas as pd
pd.DataFrame(panx_ch["en"]["train"][122]).T

,0,1,2,3,4,5,6,7,8,9,10
tokens,Jake,McGee,",",current,MLB,player,(,Tampa,Bay,Rays,)
ner_tags,1,2,0,0,3,0,0,3,4,4,0
langs,en,en,en,en,en,en,en,en,en,en,en
ner_tags_str,B-PER,I-PER,O,O,B-ORG,O,O,B-ORG,I-ORG,I-ORG,O


- ko 안의 ner 갯 수 확인

In [7]:
all_count = {}
for k in ["train", "validation", "test"]:
    all_content =[]
    for i in panx_ch['ko'][k]["ner_tags_str"]:
        for j in i:
            if "B" in j:
                all_content.append(j.split("-")[-1])
        per , org, loc =(all_content.count('PER'),
                         all_content.count('ORG'),
                         all_content.count('LOC'))
        all_count[k] = [per, org, loc]
all_count

{'train': [4097, 4495, 5845],
 'validation': [2039, 2297, 2976],
 'test': [2116, 2165, 2971]}

In [8]:
pd.DataFrame(all_count,
              index=['PER', 'ORG', 'LOC']).transpose()

,PER,ORG,LOC
train,4097,4495,5845
validation,2039,2297,2976
test,2116,2165,2971


### XLM-R 토크나이저

In [9]:
from transformers import AutoTokenizer

xlmr_model_name = "xlm-roberta-base"
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [10]:
' '.join(panx_ch['ko']['train'][1266]["tokens"])

'넘겨주기 무명 용사의 무덤'

In [11]:
xlmr_tokenizer(' '.join(panx_ch['ko']['train'][4]["tokens"]))


{'input_ids': [0, 11873, 2293, 6, 166637, 71393, 16907, 12286, 80169, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

### 모델 생성

In [12]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel, RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels # 라벨 수
        # 모델 바디 로드
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # 분류 헤드
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # 가중치 초기화
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, labels=None,
                token_type_ids=None, **kwargs):
        # 인코더 아웃풋
        outputs = self.roberta(input_ids,
                               attention_mask = attention_mask,
                               token_type_ids=token_type_ids, **kwargs)
        # 아웃풋 -> 헤드
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # 손실 계산
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # 모델 출력 반환
        return TokenClassifierOutput(loss=loss, logits=logits,
                                     hidden_states=outputs.hidden_states,
                                     attentions=outputs.attentions)

2024-11-25 17:04:33.326184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732521873.371438   68184 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732521873.385261   68184 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 17:04:33.484245: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,
                                         num_labels = len(labels),
                                         id2label = i2l,
                                         label2id = l2i)

In [14]:
import torch

device = torch.device("cuda")
xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, config=xlmr_config)
              .to(device))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def tag_text(text, tags, model, tokenizer):
    # 토크나이징
    tokens = tokenizer(text).tokens()
    
    # seq 2 input_ids
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)

    # logits
    outputs = model(input_ids)[0]

    # 가장 확율높은 클래스 선택
    predictions = torch.argmax(outputs, dim=2)

    # to df
    preds = [tags[p] for p in predictions[0]]
    return pd.DataFrame([tokens, preds], index=["tokens","tags"])

tag_text("this is test, here", labels, xlmr_model, xlmr_tokenizer)

,0,1,2,3,4,5,6
tokens,<s>,▁this,▁is,▁test,",",▁here,</s>
tags,O,O,O,O,O,O,O


In [16]:
panx_ch['ko']['train']

Dataset({
    features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
    num_rows: 10000
})

In [17]:
# 토크나이징, 어텐션 마스크, 라벨 정리
def tokenize_and_align_labels(examples):
    tokenized_input = xlmr_tokenizer(examples["tokens"], truncation=True,
                                     is_split_into_words=True)

    labels=[]
    for idx, label in enumerate(examples["ner_tags"]):
        # print(examples['tokens'][idx],label)
        word_ids = tokenized_input.word_ids(batch_index=idx)
        # print(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # print(word_idx)
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)
    tokenized_input["labels"] = labels
    return tokenized_input

In [18]:
# 모델 인풋값 정제하기
def encode_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True,
                      remove_columns=["langs", "ner_tags", "tokens", "ner_tags_str"])

In [19]:
encoded_ko =encode_dataset(panx_ch["ko"])
encoded_ko

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
})

### XLM-RoBERTa 파인튜닝 하기

In [20]:
from transformers import TrainingArguments

num_epoch = 3
batch_size = 24
logging_steps = len(encoded_ko["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-ko"
training_args = TrainingArguments(
    output_dir=model_name,
    log_level="error",
    num_train_epochs=num_epoch,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=True
    )

/home/tommy/anaconda3/envs/transformer/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# from huggingface_hub import notebook_login

# notebook_login()

In [22]:
from seqeval.metrics import f1_score
import numpy as np

def align_preds(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    label_list, preds_list = [],[]

    for batch_idx in range(batch_size):
        example_labels, example_preds = [],[]
        for seq_idx in range(seq_len):
            if label_ids[batch_idx, seq_idx] != -100: # <s> 및 </s> 무시
                example_labels.append(i2l[label_ids[batch_idx][seq_idx]])
                example_preds.append(i2l[preds[batch_idx][seq_idx]])
        label_list.append(example_labels)
        preds_list.append(example_preds)
    
    return preds_list, label_list

def compute_metrics(eval_pred):
    y_pred, y_true = align_preds(eval_pred.predictions,
                                 eval_pred.label_ids)
    return {"f1_score" : f1_score(y_pred,y_true)}

- 데이터 콜레이터(Data Collator)
    - 데이터 콜레이터는 데이터셋 요소들의 리스트를 입력으로 사용하여 배치를 형성하는 객체.
    - 이러한 요소들은 train_dataset 또는 eval_dataset의 요소들과 동일한 타입.
    - 배치를 구성하기 위해 데이터 콜레이터는 패딩과 같은 처리를 적용할 수 있다.
    - DataCollatorForLanguageModeling과 같은 일부 콜레이터는 형성된 배치에 무작위 마스킹과 같은 일부 무작위 데이터 증강을 적용.

In [23]:
from transformers import DataCollatorForTokenClassification # 입력과 레이블 패딩 전용 콜레이터

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

In [24]:
# model_init 함수 : 모델 재사용을 위한 모델 생성 함수정의
def model_init():
    return (XLMRobertaForTokenClassification
            .from_pretrained(xlmr_model_name, config=xlmr_config)
            .to(device))

In [26]:
# 모든 객체를 trainer 에 전달
from transformers import Trainer

trainer = Trainer(model_init=model_init,
                  args=training_args,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  train_dataset=encoded_ko["train"],
                  eval_dataset=encoded_ko["validation"],
                  tokenizer = xlmr_tokenizer)

trainer.train()

/tmp/ipykernel_68184/2119604388.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model_init=model_init,


Epoch,Training Loss,Validation Loss,F1 Score
1,0.454700,0.271479,0.821335
2,0.230900,0.244721,0.847851
3,0.148400,0.226946,0.864984


TrainOutput(global_step=1251, training_loss=0.2776064072533858, metrics={'train_runtime': 501.7434, 'train_samples_per_second': 59.792, 'train_steps_per_second': 2.493, 'total_flos': 835350415434480.0, 'train_loss': 0.2776064072533858, 'epoch': 3.0})

In [27]:
# trainer.push_to_hub(commit_message="first training")

In [28]:
# 테스트
test_massage = "안녕하세요 저는 서울에 살고 있습니다.."
tag_text(test_massage,
         labels, trainer.model, xlmr_tokenizer)

,0,1,2,3,4,5,6,7,8,9
tokens,<s>,▁안녕하세요,▁저는,▁서울,에,▁살고,▁있습니다,.,.,</s>
tags,O,O,O,B-LOC,B-LOC,O,O,O,O,O


In [29]:
from torch.nn.functional import cross_entropy

# 전체 데이터셋에 대한 분석
def forward_pass_with_label(batch):
    # batch to dict
    features = [dict(zip(batch,t)) for t in zip(*batch.values())]
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    # 추론 모드
    with torch.no_grad():
        output = trainer.model(input_ids, attention_mask)
        predicted_label = torch.argmax(output.logits, dim=-1).cpu().numpy()

    # 배치마다 손실 계산
    # 하나의 토큰마다 7개의 확률이 나오므로 각 출력마다 하나의 텐서로
    # 즉 정답 라벨 1개, 출력값 7개 에 대한 cross entropy loss 값이 나온다.
    loss = cross_entropy(output.logits.view(-1,7),
                         labels.view(-1), reduction="none")

    # loss 계산후 각 시퀀스별 묶기
    loss = loss.view(len(input_ids), -1).cpu().numpy()
    return {"loss":loss, "predicted_label": predicted_label}

In [30]:
# 검증 데이터 배치별 매핑
valid_result = encoded_ko['validation'].map(forward_pass_with_label, batch_size=32, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [31]:
i2l[-100] = "IGN" # 할당 <s>,</s>

In [32]:
df_result = pd.DataFrame(valid_result)

In [33]:
# 다시 index to token
df_result["input_tokens"] = df_result["input_ids"].apply(
    lambda x : xlmr_tokenizer.convert_ids_to_tokens(x))

In [34]:
df_result["predicted_label"] = df_result["predicted_label"].apply(
    lambda x : [i2l[i] for i in x])

df_result["labels"] = df_result["labels"].apply(
    lambda x : [i2l[i] for i in x])

In [ ]:
# 원래 크기에 맞춰서 패딩 자르기
df_result['loss'] = df_result.apply(
    lambda x : x['loss'][:len(x['input_ids'])], axis=1)

df_result["predicted_label"] = df_result.apply(
    lambda x : x["predicted_label"][:len(x['input_ids'])], axis=1)

In [36]:
df_result

,input_ids,attention_mask,labels,loss,predicted_label,input_tokens
0,"[0, 6339, 16295, 2099, 7286, 3999, 769, 96897,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[IGN, O, O, O, O, O, O, B-LOC, B-LOC, I-LOC, I...","[0.0, 0.0008671099785715342, 0.001348301419056...","[O, O, O, O, O, O, O, B-LOC, B-LOC, I-LOC, I-L...","[<s>, ▁지, 청, 사, 무, 소, 는, ▁아마, 미, ▁, 시에, ▁위치한, ..."
1,"[0, 61805, 23854, 167427, 11031, 7286, 6521, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[IGN, O, O, O, B-ORG, B-ORG, I-ORG, I-ORG, I-O...","[0.0, 0.00034791138023138046, 0.00037222131504...","[O, O, O, O, B-ORG, B-ORG, I-ORG, I-ORG, I-ORG...","[<s>, ▁넘, 겨, 주기, ▁상, 무, ▁신, 협, ▁배, 구, 단, </s>]"
2,"[0, 7063, 8333, 123870, 15, 106, 6192, 6780, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[IGN, B-PER, B-PER, B-PER, O, O, O, O, O, O, O...","[0.0, 0.006986474618315697, 0.0048017664812505...","[O, B-PER, B-PER, B-PER, O, O, O, O, O, O, O, O]","[<s>, ▁정, 형, 돈, ▁(, ▁1, 회, ▁~, ▁24, 회, ▁), </s>]"
3,"[0, 79978, 96648, 28185, 63993, 697, 97670, 16...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[IGN, O, O, O, O, O, O, O, O, O, B-ORG, B-ORG,...","[0.0, 0.0003415954706724733, 0.000439785566413...","[O, O, O, O, O, O, O, O, O, O, B-ORG, B-ORG, B...","[<s>, ▁그의, ▁시즌, ▁첫, ▁골, 은, ▁3-0, 으로, ▁이, 긴, ▁,..."
4,"[0, 242, 5106, 23526, 12057, 5106, 242, 769, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[IGN, O, O, B-LOC, B-LOC, O, O, O, B-LOC, B-LO...","[0.0, 0.00037126801908016205, 0.00037567710387...","[O, O, O, O, O, O, O, O, B-LOC, B-LOC, B-LOC, ...","[<s>, ▁', ▁'', ▁우, 파, ▁'', ▁', 는, ▁러시아, ▁바, 시키..."
...,...,...,...,...,...,...
4995,"[0, 101278, 6, 106286, 367, 6, 237558, 12788, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[IGN, B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, ...","[0.0, 0.4214648902416229, 0.3684261739253998, ...","[O, B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O,...","[<s>, ▁유럽, ▁, 연합, 의, ▁, 깃, 발, 에는, ▁12, 개의, ▁별,..."
4996,"[0, 8237, 9187, 46364, 15, 33892, 84386, 713, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[IGN, B-PER, B-PER, B-PER, I-PER, I-PER, I-PER...","[0.0, 0.008504010736942291, 0.0070904060266911...","[O, B-PER, B-PER, B-PER, I-PER, I-PER, I-PER, ...","[<s>, ▁김, 학, 순, ▁(, ▁여성, 운동, 가, ▁), </s>]"
4997,"[0, 44486, 6308, 3497, 8495, 1048, 5807, 10693...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[IGN, B-PER, B-PER, B-PER, B-PER, B-PER, I-PER...","[0.0, 0.005169240292161703, 0.0046163178049027...","[O, B-PER, B-PER, B-PER, B-PER, B-PER, I-PER, ...","[<s>, ▁레, 오, 나, 르, 도, ▁보, 누, 치, ▁(, ▁34, ▁), <..."
4998,"[0, 13745, 26141, 11289, 45504, 26191, 45981, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[IGN, O, O, O, O, O, O, O, O, O, O, O, O, O, O...","[0.0, 0.003446117974817753, 0.0061222868971526...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[<s>, ▁발, 효, 하지, ▁않고, ▁화, 덕, 에, ▁구, 운, ▁, 납, 작..."


In [37]:
# 인덱스 각 토큰별 분해
df_tokens = df_result.apply(pd.Series.explode)
df_tokens = df_tokens[df_tokens["labels"] != "IGN"] # IGN은 예측 loss 에서 빼기
df_tokens

,input_ids,attention_mask,labels,loss,predicted_label,input_tokens
0,6339,1,O,0.000867,O,▁지
0,16295,1,O,0.001348,O,청
0,2099,1,O,0.001185,O,사
0,7286,1,O,0.001066,O,무
0,3999,1,O,0.001376,O,소
...,...,...,...,...,...,...
4999,2166,1,O,0.000398,O,시
4999,206276,1,O,0.000371,O,▁중단
4999,50751,1,O,0.000279,O,하였다
4999,6,1,O,0.000228,O,▁


In [39]:
(
    df_tokens.groupby("labels")[["loss"]]
    .agg(["count","mean","sum"])
    .droplevel(level=0, axis=1) # 멀티 컬럼 삭제
    .sort_values(by="sum", ascending=False)
    .reset_index()
    .head(10)
    .T
)

,0,1,2,3,4,5,6
labels,B-ORG,O,I-ORG,B-PER,B-LOC,I-PER,I-LOC
count,6505,53513,6374,6025,9839,4596,2267
mean,0.766222,0.085303,0.482868,0.454054,0.20701,0.35287,0.520431
sum,4984.272879,4564.80906,3077.798421,2735.678038,2036.774456,1621.78849,1179.816505


- B-ORG 의 평균 손실이 가장 높다

### 제로샷 전이 : 교차 언어 전이학습

In [96]:
def get_f1_score(trainer, dataset):
    return trainer.predict(dataset).metrics["test_f1_score"]

In [73]:
encoded_en = encode_dataset(panx_ch["en"])
encoded_ja = encode_dataset(panx_ch["ja"])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [109]:
get_f1_score(trainer ,encoded_en["valiation"])

0.5389431167956916

In [114]:
get_f1_score(trainer ,encoded_ja["validation"])

0.20663811563169163

In [135]:
def train_on_subset(dataset, num_samples):
    train_ds = dataset["train"].shuffle(seed=42).select(range(num_samples))
    valid_ds = dataset["validation"]
    test_ds = dataset["test"]

    training_args.logging_steps = len(train_ds) // batch_size
    trainer = Trainer(model_init=model_init,
                      args=training_args,
                      data_collator=data_collator,
                      compute_metrics=compute_metrics,
                      train_dataset=train_ds,
                      eval_dataset=valid_ds)
    
    trainer.train()
    if training_args.push_to_hub:
        trainer.push_to_hub(commit_message = "more_train")
    
    f1_score = get_f1_score(trainer, test_ds)   
    
    return pd.DataFrame.from_dict({
        "num_sample" : [len(train_ds)], "f1_score" : [f1_score]})

train_on_subset(encoded_ja, 250)

Epoch,Training Loss,Validation Loss,F1 Score
1,1.153000,1.082186,0.000000
2,0.914000,1.007886,0.000000
3,0.815800,0.922073,0.000000


events.out.tfevents.1732539057.DESKTOP-2KLMMI4.68184.3:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

,num_sample,f1_score
0,250,0.0
